In [1]:
#Importing all the neccessary pacakges
import pandas as pd
import numpy as np
import datetime

In [2]:
#Initialising the dataframes
orders_list=pd.read_csv("data/List of Orders.csv").dropna()
order_details=pd.read_csv("data/Order Details.csv").dropna()
sales_target=pd.read_csv("data/Sales target.csv").dropna()

In [3]:
orders_list.head()

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


In [4]:
order_details.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


In [5]:
sales_target.head()

,Month of Order Date,Category,Target
0,Apr-18,Furniture,10400.0
1,May-18,Furniture,10500.0
2,Jun-18,Furniture,10600.0
3,Jul-18,Furniture,10800.0
4,Aug-18,Furniture,10900.0


In [6]:
#Convert order_date column to datetime
orders_list['Order Date'] = pd.to_datetime(orders_list['Order Date'],format='mixed')
orders_list

,Order ID,Order Date,CustomerName,State,City
0,B-25601,2018-01-04,Bharat,Gujarat,Ahmedabad
1,B-25602,2018-01-04,Pearl,Maharashtra,Pune
2,B-25603,2018-03-04,Jahan,Madhya Pradesh,Bhopal
3,B-25604,2018-03-04,Divsha,Rajasthan,Jaipur
4,B-25605,2018-05-04,Kasheen,West Bengal,Kolkata
...,...,...,...,...,...
495,B-26096,2019-03-28,Atharv,West Bengal,Kolkata
496,B-26097,2019-03-28,Vini,Karnataka,Bangalore
497,B-26098,2019-03-29,Pinky,Jammu and Kashmir,Kashmir
498,B-26099,2019-03-30,Bhishm,Maharashtra,Mumbai


In [7]:
#What is the average sales made for each month based on category?
average_df=pd.merge(order_details[['Order ID','Amount','Profit']],orders_list[['Order ID','Order Date']],on='Order ID',how='inner').dropna()
average_df=average_df.rename(columns={'Order Date':'Order_Date'})
average_df['monthname']=average_df['Order_Date'].dt.month_name()
average_df['monthno']=average_df['Order_Date'].dt.month
average_df=average_df.groupby(['monthno','monthname']).mean(numeric_only=True)
average_df

,,Amount,Profit
monthno,monthname,,
1,January,326.109524,25.519048
2,February,277.283951,36.740741
3,March,261.855556,40.566667
4,April,313.016807,2.117647
5,May,243.254902,0.509804
6,June,189.072727,-27.809091
7,July,244.682353,-12.823529
8,August,289.530726,-2.597765
9,September,263.193878,-19.510204


In [8]:
# 2.Which months have the target not been made/made based on category?  
ol_lst_df=orders_list #Orders list dataframe
ol_lst_df['Month']=pd.to_datetime(ol_lst_df['Order Date']).dt.month_name().str[:3]
ol_lst_df['Year']=pd.to_datetime(ol_lst_df['Order Date']).dt.year #Creating new columns in output dataframe to create join condition

sales_tgt_df=sales_target
sales_tgt_df['Year']=pd.to_numeric('20'+sales_tgt_df['Month of Order Date'].str.split('-').str[1])
sales_tgt_df['Month']=sales_tgt_df['Month of Order Date'].str.split('-').str[0]
output_df=pd.merge(ol_lst_df[['Order ID','Month','Year']],order_details[['Order ID','Category','Amount']], \
                   how='inner',on=['Order ID'])
output_df=output_df.groupby(['Category','Month','Year'])['Amount'].sum().reset_index()
output_df=pd.merge(sales_tgt_df,output_df,how='inner',on=['Month','Year','Category'])
output_df[output_df['Amount']>output_df['Target']].reset_index(drop=True)


,Month of Order Date,Category,Target,Year,Month,Amount
0,Aug-18,Furniture,10900.0,2018,Aug,15259.0
1,Nov-18,Furniture,11300.0,2018,Nov,12465.0
2,Jan-19,Furniture,11500.0,2019,Jan,15944.0
3,Aug-18,Electronics,9000.0,2018,Aug,16740.0
4,Oct-18,Electronics,9000.0,2018,Oct,15537.0
5,Nov-18,Electronics,9000.0,2018,Nov,10531.0
6,Dec-18,Electronics,9000.0,2018,Dec,11915.0
7,Jan-19,Electronics,16000.0,2019,Jan,22637.0


In [9]:
# 3. What is the profit/loss margin made for each category? 
output_df=order_details.groupby('Category')['Profit'].agg(Profit='sum')
print(output_df)

              Profit
Category            
Clothing     11163.0
Electronics  10494.0
Furniture     2298.0


In [10]:
# 4. Which months do we see the highest sales for 2018?
output_df=pd.merge(orders_list,order_details,on=['Order ID']) 
output_df=output_df[(output_df['Order Date']>='2018-01-01') & (output_df['Order Date']<='2018-12-31')]
output_df['Month']=pd.to_datetime(output_df['Order Date']).dt.month_name().str[:3]
output_df=output_df.groupby(['Category','Month'])['Amount'].agg(Sales='sum')
output_df=output_df.groupby(['Category'])['Sales'].idxmax()
output_df


Category
Clothing          (Clothing, Nov)
Electronics    (Electronics, Aug)
Furniture        (Furniture, Aug)
Name: Sales, dtype: object

In [11]:
#5. For each category- return the subcategory performing the best in terms of sales and the subcategory with lowest number of sales/profit.
output_df=pd.merge(orders_list,order_details,on=['Order ID'])
output_df=output_df.groupby(['Category','Sub-Category'])['Amount'].agg(Highestsales='max',Minsales='min')
highestsales=output_df.groupby(['Category'])['Highestsales'].idxmax()
print(f"Category and subcategory with highest sales: \n {highestsales} \n")
lowestsales=output_df.groupby(['Category'])['Minsales'].idxmin()
print(f"Category and subcategory with lowest sales: \n {lowestsales} \n")

Category and subcategory with highest sales: 
 Category
Clothing          (Clothing, Trousers)
Electronics    (Electronics, Printers)
Furniture          (Furniture, Chairs)
Name: Highestsales, dtype: object 

Category and subcategory with lowest sales: 
 Category
Clothing                (Clothing, Skirt)
Electronics    (Electronics, Accessories)
Furniture        (Furniture, Furnishings)
Name: Minsales, dtype: object 



In [12]:
#6. Return the top 5 or top 10 customers who made the most purchases
output_df=pd.merge(orders_list,order_details,on=['Order ID'])
output_df=output_df.groupby('CustomerName')['Order ID'].agg(Orders='count')
output_df.sort_values(by='Orders',ascending=False).head(5)

,Orders
CustomerName,
Abhishek,25
Priyanka,21
Shreya,18
Pooja,17
Shruti,17


In [13]:
#7. Which states have the highest sales and their profit margins?
output_df=pd.merge(orders_list,order_details,on=['Order ID'])
output_df=output_df.groupby('State')['Order ID'].agg(Orders='count')
output_df.sort_values(by='Orders',ascending=False).head(5)

,Orders
State,
Madhya Pradesh,340
Maharashtra,290
Gujarat,87
Delhi,74
Rajasthan,74


In [39]:
#8. What has the sale,profit been over the few months and its growth percentage over time? 
output_df=pd.merge(orders_list,order_details,on=['Order ID'])
output_df['monthno']=output_df['Order Date'].dt.month
output_df=output_df[['Category','Year','Month','monthno','Amount']] \
.groupby(by=['Category','Year','Month','monthno'])['Amount'].agg(Sales='sum') \
.sort_values(by=['Category','Year','monthno']).reset_index(level=['monthno'])
output_df['Growth Rate']=output_df['Sales'].groupby(['Category']).pct_change(1)*100
output_df.drop(columns='monthno')


Sales  Growth Rate
Category  Year Month                      
Clothing  2018 Jan     5355.0          NaN
               Feb     1662.0   -68.963585
               Mar     3521.0   111.853189
               Apr    10743.0   205.112184
               May     6986.0   -34.971609
...                       ...          ...
Furniture 2019 Jul     1494.0   150.251256
               Aug     2826.0    89.156627
               Sep     1271.0   -55.024770
               Oct     7819.0   515.184894
               Nov      421.0   -94.615680

[71 rows x 2 columns]